In [11]:
#%pip install -q --upgrade crewai langchain-openai python-dotenv openai gradio

In [12]:
#pip install gradio

In [13]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from openai import AzureOpenAI
import os
import requests
from datetime import datetime
import yaml
from dotenv import load_dotenv
import pyodbc
import gradio as gr

In [14]:
# load_dotenv()

# # Configuration - loaded from .env file
# AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
# AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
# AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
# AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

# # Validate configuration early for clearer errors
# missing = [
#     name for name, val in [
#         ("AZURE_OPENAI_ENDPOINT", AZURE_OPENAI_ENDPOINT),
#         ("AZURE_OPENAI_API_KEY", AZURE_OPENAI_API_KEY),
#         ("AZURE_OPENAI_DEPLOYMENT", AZURE_OPENAI_DEPLOYMENT),
#         ("AZURE_OPENAI_API_VERSION", AZURE_OPENAI_API_VERSION),
#     ]
#     if not val
# ]
# if missing:
#     raise ValueError(
#         f"Missing required environment variables: {', '.join(missing)}.\n"
#         "Create a .env file with these values or set them in your environment."
#     )

# # Configure CrewAI LLM for Azure OpenAI via OpenAI-compatible endpoint
# llm = LLM(
#     model=AZURE_OPENAI_DEPLOYMENT,
#     api_key=AZURE_OPENAI_API_KEY,
#     base_url=f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT}",
#     default_headers={"api-key": AZURE_OPENAI_API_KEY},
#     default_query={"api-version": AZURE_OPENAI_API_VERSION},
#     provider="openai",
# )

In [15]:
# Initialize Ollama LLM using CrewAI's LLM wrapper
llm = LLM(model="ollama/llama3.2", base_url="http://lambda2.uncw.edu:11434/api/generate")


In [18]:
# Azure SQL connection
conn_str = (
    f"Driver={{ODBC Driver 18 for SQL Server}};"
    f"Server={os.getenv('AZURE_SQL_SERVER')};"
    f"Database={os.getenv('AZURE_SQL_DATABASE')};"
    f"Uid={os.getenv('AZURE_SQL_USER')};"
    f"Pwd={os.getenv('AZURE_SQL_PASSWORD')};"
    f"Encrypt=yes;TrustServerCertificate=no;"
)

def query_db(query):
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()

# Agent
analyst = Agent(
    role='IMDb Database Analyst',
    goal='Analyze IMDb movie data and provide insights',
    backstory='Expert SQL analyst specializing in movie data',
    llm=llm,
    verbose=True
)

# Task
task = Task(
    description="""
    Query the IMDb database to find:
    # 1. Top 10 highest rated movies
    # 2. Most prolific directors
    
    # Use SQL: SELECT title, rating FROM movies ORDER BY rating DESC LIMIT 10
    """,
    agent=analyst,
    expected_output='Analysis report with top movies and directors'
)

# Run
crew = Crew(agents=[analyst], tasks=[task], verbose=True)
result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ff6308bd-934c-4765-90d5-be031c91b30d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: IMDb Database Analyst                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Query the IMDb database to find:                                                                           │
│      # 1. Top 10 highest rated movies                                                                           │
│      # 2. Most prolific directors                                                                               │
│                                                                                                                 │
│      # Use SQL: SELECT title, rating FROM movies ORDER BY rating DESC LIMIT 10                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: IMDb Database Analyst                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Top 10 Highest Rated Movies:                                                                                   │
│                                                                                                                 │
│  SELECT title, rating FROM movies ORDER BY rating DESC LIMIT 10;                                                │
│                                                                                                                 │
│  Most Prolific Directors:                                                                                       │
│                                                                                                                 │
│  SELECT name, COUNT(titles) AS num_movies                                                                       │
│  FROM directors                                                                                                 │
│  GROUP BY name                                                                                                  │
│  ORDER BY num_movies DESC                                                                                       │
│                                                                                                                 │
│  --Note: The above SQL query returns the number of movies each director has directed.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4a0aecdb-a5be-47cf-8d0d-09c291ea2f43                                                                     │
│  Agent: IMDb Database Analyst                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Gradio Interface
A simple chat-style UI to query the IMDb database using natural language. The agent will interpret your question, generate the appropriate SQL, and return the results.

In [19]:
def run_query(user_question: str) -> str:
    """Create a fresh task from the user's question and run the crew."""
    if not user_question.strip():
        return "Please enter a question."

    query_task = Task(
        description=f"""
        The user asked: "{user_question}"

        Query the IMDb SQL database to answer this question.
        Write and execute the appropriate SQL query, then summarise the results
        in a clear, readable format.
        """,
        agent=analyst,
        expected_output="A clear answer to the user's question based on the database results.",
    )

    query_crew = Crew(agents=[analyst], tasks=[query_task], verbose=False)

    try:
        result = query_crew.kickoff()
        return str(result)
    except Exception as e:
        return f"Error: {e}"


# --- Gradio UI ---
EXAMPLE_QUESTIONS = [
    ["What are the top 10 highest rated movies?"],
    ["Who are the most prolific directors?"],
    ["Which genres have the highest average rating?"],
    ["How many movies were released each decade?"],
]

with gr.Blocks(title="IMDb AI Analyst") as demo:
    gr.Markdown("# 🎬 IMDb AI Analyst")
    gr.Markdown("Ask any question about the IMDb database and the AI agent will query it for you.")

    with gr.Row():
        question_box = gr.Textbox(
            label="Your Question",
            placeholder="e.g. What are the top 10 highest rated movies?",
            lines=2,
            scale=4,
        )
        submit_btn = gr.Button("Ask", variant="primary", scale=1)

    answer_box = gr.Textbox(label="Answer", lines=12, interactive=False)

    gr.Examples(examples=EXAMPLE_QUESTIONS, inputs=question_box)

    submit_btn.click(fn=run_query, inputs=question_box, outputs=answer_box)
    question_box.submit(fn=run_query, inputs=question_box, outputs=answer_box)

demo.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: IMDb Database Analyst                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│          The user asked: "What are the top ten movies with dogs?"                                               │
│                                                                                                                 │
│          Query the IMDb SQL database to answer this question.                                                   │
│          Write and execute the appropriate SQL query, then summarise the results                                │
│          in a clear, readable format.                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: IMDb Database Analyst                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The following are the top ten movies with dogs based on IMDb ratings:                                          │
│                                                                                                                 │
│  | Rank | Title (Original Title)        | Release Year | Runtime  | Genre          | Rating |                   │
│  |------|-------------------------------|--------------|---------|----------------|--------|                    │
│  |    1 | Hachiko: A Dog's Story         | 2009         | 100     | Drama, Family   | 7.5/10  |                 │
│  |    2 | Marley & Me                   | 2008         | 105     | Comedy, Drama   | 7.4/10  |                  │
│  |    3 | Beethoven                     | 1992         | 91      | Comedy        | 7.1/10  |                    │
│  |    4 | Turner & Hooch                 | 1989         | 110     | Comedy, Crime  | 7.0/10  |                  │
│  |    5 | Best in Show                    | 2000         | 95      | Comedy        | 6.9/10  |                  │
│  |    6 | Air Bud                        | 1997         | 80      | Family, Sport | 6.8/10  |                   │
│  |    7 | Uggie & Hattie                  | 2011         | 90      | Documentary   | 6.7/10  |                  │
│  |    8 | Eight Below                     | 2006         | 100     | Drama, Family   | 6.6/10  |                │
│  |    9 | My Dog Skip                     | 2000         | 91      | Family, Drama  | 6.5/10  |                 │
│  |   10 | The Shaggy Dog                   | 2006         | 106     | Comedy        | 6.4/10  |                 │
│                                                                                                                 │
│  Note: IMDb ratings are subject to change over time and may not reflect the current rating.                     │
│                                                                                                                 │
│  These movies are ranked based on their average IMDb rating, with Hachiko: A Dog's Story topping the list as    │
│  the highest rated film featuring dogs.                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

demo.close()